In [1]:
import os#环境代理设置
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import Image
# default: 100
mpl.rcParams['figure.dpi'] = 150
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

gpt</br>
| model        | 参数量 | hidden dim     | block 数量 |
|--------------|--------|----------------|------------|
| gpt2         | 124M   | 768 (64×12)    | 12         |
| gpt2-medium  | 355M   | 1024 (64×16)   | 24         |
| gpt2-large   | 774M   | 1280 (64×20)   | 36         |
| gpt2-xl      | 1.56B  | 1600 (64×25)   | 48         |

In [3]:
from transformers import AutoModelForCausalLM
from transformers_utils import get_params

In [4]:
model_ckpt = 'gpt2-xl'
config = AutoConfig.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

/home/jiangzirou/data/miniconda3/envs/PyTroch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [5]:
# model_clm = AutoModelForCausalLM.from_pretrained(model_ckpt)
print(format(get_params(model), ','))

1,557,611,200


In [6]:
config

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 50257
}

In [7]:
model

GPT2Model(
  (wte): Embedding(50257, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-47): 48 x GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=4800, nx=1600)
        (c_proj): Conv1D(nf=1600, nx=1600)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=6400, nx=1600)
        (c_proj): Conv1D(nf=1600, nx=6400)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
)

tokenizer

In [8]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2-xl', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [9]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [10]:
tokenizer.special_tokens_map_extended

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [11]:
tokenizer.encode('<|endoftext|>')

[50256]

In [12]:
tokenizer.decode(50256)

'<|endoftext|>'

In [13]:
# 大小写敏感
print(tokenizer.encode('Hello'))
print(tokenizer.encode('hello'))
print(tokenizer.encode(' hello'))
print(tokenizer.decode(23748))
print(tokenizer.encode(' hello'))
print(tokenizer.encode('  hello'))
print(tokenizer.encode('   hello'))

[15496]
[31373]
[23748]
 hello
[23748]
[220, 23748]
[220, 220, 23748]


In [14]:
tokenizer('My favorite color is chartreuse')

{'input_ids': [3666, 4004, 3124, 318, 8262, 260, 1904], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.encode(' chartreuse')

[8262, 260, 1904]

### attention_mask
- 更好地构造结构化，批次化输入（tensor，shape一定）

In [16]:
tokenizer.pad_token = tokenizer.eos_token
sentences = ["It will rain in the",
            "I want to eat a big bowl of",
            "My dog is"]
inputs = tokenizer(sentences, return_tensors="pt", padding=True)

In [17]:
inputs

{'input_ids': tensor([[ 1026,   481,  6290,   287,   262, 50256, 50256, 50256],
        [   40,   765,   284,  4483,   257,  1263,  9396,   286],
        [ 3666,  3290,   318, 50256, 50256, 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0]])}

### forward
- GPT2Model
    - wte: word token embedding
    - wpe: word position embedding
- LMHead
    - mlp: hidden_state => vocab_size

In [18]:
model_ckpt = 'gpt2-xl'
config = AutoConfig.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
model_clm = AutoModelForCausalLM.from_pretrained(model_ckpt).to(device)

In [19]:
model

GPT2Model(
  (wte): Embedding(50257, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-47): 48 x GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=4800, nx=1600)
        (c_proj): Conv1D(nf=1600, nx=1600)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=6400, nx=1600)
        (c_proj): Conv1D(nf=1600, nx=6400)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
)

model_clm(GPT2LMHeadModel)
- GPT2Model
- lm_head

In [20]:
model_clm 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=4800, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=1600)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6400, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=6400)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)

In [21]:
# 大小写是不同的token，
# 空格也是token
input_txt = "A long long time ago"
model_inputs = tokenizer(input_txt, return_tensors='pt')
model_inputs

{'input_ids': tensor([[  32,  890,  890,  640, 2084]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [22]:
input_ids = model_inputs['input_ids'].to(device)

In [23]:
output = model_clm(input_ids=input_ids)

In [24]:
output.logits.shape

torch.Size([1, 5, 50257])

model.transformer()</br>
`(transformer): GPT2Model`

- 设为评估模式.eval()
    - Dropout 层：在训练模式下会随机丢弃部分神经元，而在评估模式下会保留所有神经元。
    - BatchNorm 层：在训练模式下会使用当前 batch 的均值和方差，而在评估模式下会使用训练阶段累计的均值和方差。

In [25]:
model_clm.eval() 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=4800, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=1600)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6400, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=6400)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)

In [26]:
outputs = model_clm.transformer(input_ids)

In [27]:
outputs.last_hidden_state

tensor([[[-0.5478,  0.0327,  0.7692,  ..., -4.5986,  0.2638,  0.1780],
         [-1.0570, -0.8408,  0.8223,  ..., -1.3309,  1.0921,  0.7939],
         [-0.5805, -1.1760,  0.4954,  ..., -1.3462,  1.2957,  1.2741],
         [-0.2789, -1.5347,  0.6579,  ..., -1.6087,  1.1029,  0.9785],
         [ 0.2617,  0.0104,  0.6141,  ..., -1.6620,  1.4646,  0.6588]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [28]:
outputs.last_hidden_state.shape

torch.Size([1, 5, 1600])

自定义（model.transformer()）

input_ids = input_ids.view(-1, input_shape[-1])  
- [4, 2, 512] ->[8, 512]
- 为了把多个“选择”或“样本”展平成一个批次，以便可以一次性喂给模型处理，提高效率。

In [ ]:
def gpt2_transformer_forward(model, input_ids):
    print("input_ids.shape:",input_ids.shape)
    model.eval()
    input_shape = input_ids.size()
    # (batch_size*seq_len, hidden_size)
    input_ids = input_ids.view(-1, input_shape[-1])  
    batch_size = input_ids.shape[0]

    # 上一回合缓存的kv-cache
    past_key_values = tuple([None]*len(model.h)) # (None)*48, dtype = tuple
    
    # 构造位置 id
    # 从past_length开始，长为hidden_size的一维向量，并转换为二维向量
    past_length = 0
    position_ids = torch.arange(past_length, input_shape[-1]+past_length, dtype=torch.long, device=device)
    position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

    encoder_attention_mask = None

    # embedding
    input_embeds = model.wte(input_ids)
    position_embeds = model.wpe(position_ids)
    hidden_states = input_embeds + position_embeds
    output_shape = input_shape + (hidden_states.size(-1),)
    print("output_shape:",output_shape)

    head_mask = model.get_head_mask(None, model.config.n_layer)# [None]*48, dtype = list
    print("head_mask:",head_mask)

    for i, (block, layer_past) in enumerate(zip(model.h, past_key_values)):
        print(f"{i}-th block:")
        print("hidden_states:\n",hidden_states)
        print("layer_past:",layer_past)
        print("head_mask:",head_mask[i])

        outputs = block(
            hidden_states,
            layer_past=layer_past,
            head_mask=head_mask[i]
        )
        # 这一层输出当作下一层输入
        hidden_states = outputs[0]
        print("hidden_states:",hidden_states.shape)
    # 最终输出
    hidden_states = model.ln_f(hidden_states)
    hidden_states = hidden_states.view(output_shape)
    return hidden_states

with torch.no_grad():
    result = gpt2_transformer_forward(model_clm.transformer, input_ids)

input_ids.shape: torch.Size([1, 5])
output_shape: torch.Size([1, 5, 1600])
head_mask: [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
0-th block:
hidden_states:
 tensor([[[ 0.0561, -0.0133, -0.0379,  ...,  0.0304,  0.1525,  0.0057],
         [-0.1368,  0.0144,  0.0026,  ...,  0.0229, -0.2616, -0.0106],
         [-0.1034,  0.0086,  0.0022,  ..., -0.0359, -0.1381, -0.0129],
         [-0.0525, -0.0773, -0.0579,  ..., -0.0693, -0.1473,  0.0524],
         [ 0.0348, -0.0597,  0.0203,  ...,  0.0646, -0.0895,  0.0079]]],
       device='cuda:0')
layer_past: None
head_mask: None
hidden_states: torch.Size([1, 5, 1600])
1-th block:
hidden_states:
 tensor([[[ 0.2438,  0.3277,  0.0373,  ...,  0.6353, -0.1278, -0.0722],
         [ 0.0763,  0.4924,  0.3183,  ..., -

In [44]:
result.shape

torch.Size([1, 5, 1600])

lmhead_model(input_ids)
- logits
- past_key_values

In [66]:
model_clm(input_ids).logits

tensor([[[ 1.5445,  2.5812,  0.6197,  ..., -7.4777, -5.5282,  0.6108],
         [ 1.0344,  1.5582, -2.5840,  ..., -6.3941, -2.5380, -1.0215],
         [ 2.0070,  2.3330, -1.1671,  ..., -7.7938, -3.9385,  1.0949],
         [ 5.6171,  4.4350, -2.1919,  ..., -7.9733, -5.2310,  1.6343],
         [ 4.8450,  5.3529, -0.7294,  ..., -7.7071, -3.8473,  2.5437]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

自定义 lmhead_model forward

In [71]:
def gpt2_clm_forward(model, input_ids):
    model.eval()
    transformer_outputs = model.transformer(input_ids)
    hidden_states = transformer_outputs[0]
    print("hidden_states:",hidden_states.shape)
    logits = model.lm_head(hidden_states)
    print("logits:",logits.shape)
    return logits

model_clm.transformer输出
- last_hidden_state
- past_key_values

In [73]:
gpt2_clm_forward(model_clm, input_ids)

hidden_states: torch.Size([1, 5, 1600])
logits: torch.Size([1, 5, 50257])


tensor([[[ 1.5445,  2.5812,  0.6197,  ..., -7.4777, -5.5282,  0.6108],
         [ 1.0344,  1.5582, -2.5840,  ..., -6.3941, -2.5380, -1.0215],
         [ 2.0070,  2.3330, -1.1671,  ..., -7.7938, -3.9385,  1.0949],
         [ 5.6171,  4.4350, -2.1919,  ..., -7.9733, -5.2310,  1.6343],
         [ 4.8450,  5.3529, -0.7294,  ..., -7.7071, -3.8473,  2.5437]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)